In [ ]:
import zipfile, os, cv2
import numpy as np
import tensorflow as tf
tf.executing_eagerly()
import tensorflow_addons as tfa
from tensorflow import keras
# Display
# from IPython.display import Image
from keras import backend as K
import gc 
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
K.clear_session()

In [ ]:
cv=1

In [ ]:
save_cv_data_path= '../nii_save_np/lacune/'

np_image = np.load(save_cv_data_path+f'cv{cv}_easy_x_strain_val.npy')
np_masks = np.load(save_cv_data_path+f'cv{cv}_easy_y_strain_val.npy')
path = np.load(save_cv_data_path+f'cv{cv}_easy_path_train_val.npy')
print(np_image.shape, np_masks.shape, path.shape)

print(np_image.shape, np_masks.shape)

In [ ]:
ds=int(np_image.shape[0]*0.8)# data_scale
ds

In [ ]:
x_train = np_image[:ds].astype(np.float32)
y_train = np_masks[:ds]
x_val = np_image[ds:].astype(np.float32)
y_val = np_masks[ds:]
path_train = path[ds:]
path_val = path[:ds]
x_train = np.reshape(x_train, (x_train.shape[0]*x_train.shape[1],192,192,1))
y_train = np.reshape(y_train, (y_train.shape[0]*y_train.shape[1],192,192,1))
x_val = np.reshape(x_val, (x_val.shape[0]*x_val.shape[1],192,192,1))
y_val = np.reshape(y_val, (y_val.shape[0]*y_val.shape[1],192,192,1))
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

In [ ]:
def del_darkimg(img,mask):
    temp_i = []
    temp_m = []
    for i in range(img.shape[0]):
        if np.sum(img[i])!=0:
            temp_i.append(img[i])
            temp_m.append(mask[i])
    temp = np.array(temp_i)
    temp2 = np.array(temp_m)
    return temp, temp2
X_train, Y_train = del_darkimg(x_train, y_train)
X_val, Y_val = del_darkimg(x_val, y_val)
print(X_train.shape, Y_train.shape, X_val.shape, Y_val.shape)

In [ ]:
# data augmentation and data zip
from ImageDataAugmentor.image_data_augmentor import *
import albumentations as A

AUGMENTATIONS = A.Compose([
    A.Rotate(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(p=0.5)
],)  



# dataloaders
img_data_gen = ImageDataAugmentor(augment=AUGMENTATIONS, input_augment_mode='image', seed=123)
mask_data_gen = ImageDataAugmentor(augment=AUGMENTATIONS, input_augment_mode='mask',seed=123,)

image_data_valid = ImageDataAugmentor(augment=None, input_augment_mode='image',seed=123)
mask_data_valid = ImageDataAugmentor(augment=None, input_augment_mode='mask',seed=123)

batch_size=32
X_train_datagen = img_data_gen.flow(X_train, batch_size=batch_size)
Y_train_datagen = mask_data_gen.flow(Y_train, batch_size=batch_size)
train_generator = zip(X_train_datagen, Y_train_datagen)

X_valid_datagen = image_data_valid.flow(X_val, batch_size=batch_size)
Y_valid_datagen = mask_data_valid.flow(Y_val, batch_size=batch_size)
valid_generator = zip(X_valid_datagen, Y_valid_datagen)

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))

def DBCEL(targets, inputs, smooth=1e-6):
    dice_loss = 1 - dice_coef(targets, inputs)
    BCE =  binary_crossentropy(targets, inputs)
    DBCE = BCE + dice_loss
    
    return DBCE

def FTL(targets, inputs, alpha=0.3, beta=0.7, gamma=2, smooth=1e-6):
    
        #flatten label and prediction tensors
        inputs = K.flatten(inputs)
        targets = K.flatten(targets)
        
        #True Positives, False Positives & False Negatives
        TP = K.sum((inputs * targets))
        FP = K.sum(((1-targets) * inputs))
        FN = K.sum((targets * (1-inputs)))
               
        Tversky = (TP + smooth) / (TP + alpha*FP + beta*FN + smooth)  
        FocalTversky = K.pow((1 - Tversky), gamma)
        return FocalTversky
loss = DBCEL

In [ ]:
%env SM_FRAMEWORK=tf.keras
from segmentation_models import Unet
import segmentation_models as sm
from keras.optimizers import Adam
from tensorflow_addons.optimizers import AdamW
from keras import backend as K
from keras.metrics import binary_crossentropy
# fitting shape [[slice, w, h, c], class]
img_rows=192
img_cols=192
lr = 1e-3
epochs = 80


In [ ]:
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, LearningRateScheduler
weight_dir = 'checkpoint/seg_package/'
checkpoint_name = 'test'
# checkpoint_name = f'154_2DDense121_binary_batch-32_DBCEL_leaky_aug_norm-01_LRS_Adam-lr-{lr}_cv{cv}_1'
logdir = os.path.join("checkpoint/seg_package_tensorboard/", checkpoint_name)

In [ ]:
# Define Learning Rate Scheduler
class SnapshotCallbackBuilder:
    def __init__(self, nb_epochs, nb_snapshots, init_lr=0.1):
        self.T = nb_epochs
        self.M = nb_snapshots
        self.alpha_zero = init_lr

    def get_callbacks(self, model_prefix='Model'):

        callback_list = [
            ModelCheckpoint(os.path.join(weight_dir,f"{checkpoint_name}.hdf5"), 
                                   monitor='val_loss', mode="min", verbose=1, save_best_only=True),
            TensorBoard(log_dir=logdir,histogram_freq=1,
                                         embeddings_freq=0,embeddings_layer_names=None,),
#             EarlyStopping(monitor="val_loss", patience=15, verbose=1),
#             swa,
            LearningRateScheduler(schedule=self._cosine_anneal_schedule)
        ]

        return callback_list

    def _cosine_anneal_schedule(self, t):
        cos_inner = np.pi * (t % (self.T // self.M))  # t - 1 is used when t has 1-based indexing.
        cos_inner /= self.T // self.M
        cos_out = np.cos(cos_inner) + 1
        return float(self.alpha_zero / 2 * cos_out)

# Stochastic Weight Averaging
class SWA(keras.callbacks.Callback):
    
    def __init__(self, filepath, swa_epoch):
        super(SWA, self).__init__()
        self.filepath = filepath
        self.swa_epoch = swa_epoch 
    
    def on_train_begin(self, logs=None):
        self.nb_epoch = self.params['epochs']
        print('Stochastic weight averaging selected for last {} epochs.'
              .format(self.nb_epoch - self.swa_epoch))
        
    def on_epoch_end(self, epoch, logs=None):
        
        if epoch == self.swa_epoch:
            self.swa_weights = self.model.get_weights()
            
        elif epoch > self.swa_epoch:    
            for i in range(len(self.swa_weights)):
                self.swa_weights[i] = (self.swa_weights[i] * 
                    (epoch - self.swa_epoch) + self.model.get_weights()[i])/((epoch - self.swa_epoch)  + 1)  

        else:
            pass
        
    def on_train_end(self, logs=None):
        self.model.set_weights(self.swa_weights)
        print('Final model parameters set to stochastic weight average.')
        self.model.save_weights(self.filepath)
        print('Final stochastic averaged weights saved to file.')

In [ ]:
snapshot = SnapshotCallbackBuilder(nb_epochs=epochs,nb_snapshots=1,init_lr=lr)
swa = SWA(weight_dir + checkpoint_name+'_swa', int(epochs*0.9))

model = Unet(backbone_name='densenet121', encoder_weights=None, input_shape=(img_rows, img_cols, 1))

optimizer = 'adam'
model.compile(optimizer=optimizer, 
              loss=loss, 
              metrics=['accuracy'
                       ,dice_coef
                       ,tf.keras.metrics.MeanIoU(num_classes=2, name='iou')
                       ,tf.keras.metrics.AUC()
                       ,tf.keras.metrics.Recall()])

print(weight_dir+checkpoint_name)
# model.summary()

In [ ]:
print('-'*30,'\nFitting model...\n','-'*30)

history = model.fit(train_generator, epochs=epochs, 
                    verbose=1,
                    steps_per_epoch= len(y_train)/32*5,
                    shuffle=True, validation_data=valid_generator,
                    validation_steps= len(y_val)/32,          
                    callbacks=snapshot.get_callbacks()
                   )

In [ ]:
print('-'*30)
print('Loading and preprocessing test data...')
print('-'*30)


imgs_test = X_val
imgs_mask_test = Y_val


print('-'*30)
print('Loading saved weights...')
print('-'*30)

model = Unet(backbone_name='densenet121', encoder_weights=None, input_shape=(img_rows, img_cols, 1))
checkpoint_names = weight_dir+checkpoint_name + '.hdf5'
print(checkpoint_names)
model.load_weights(checkpoint_names)

In [ ]:
Results = model.predict(imgs_test, batch_size=1, verbose=1)

In [ ]:
print(Results.shape)
# print(np.array(sorted(path_val)))

In [ ]:
import io
import requests
flatten_pred = np.reshape(Results,(Results.shape[0]*192*192,Results.shape[-1]))
flatten_truth = np.reshape(imgs_mask_test,(imgs_mask_test.shape[0]*192*192,imgs_mask_test.shape[-1]))

In [ ]:
# print(flatten_pred.shape, flatten_truth.shape)
argmax_pred = np.reshape((flatten_pred>0.10).astype(int),(len(flatten_pred)))
argmax_truth = np.reshape(flatten_truth, (flatten_truth.shape[0])).astype(int)
# print(argmax_pred[0:5], argmax_truth[0:5])

In [ ]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
cm = confusion_matrix(argmax_truth, argmax_pred)
np.set_printoptions(precision=2)
print(cm)

In [ ]:
import matplotlib.pyplot as plt
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=25)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90,fontsize=15)
    plt.yticks(tick_marks, classes,fontsize=15)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black",
                 fontsize=20)
    
    plt.ylabel('True label', fontsize=20)
    plt.xlabel('Predicted label',fontsize=20)
    plt.tight_layout()

In [ ]:
species = ['Non-Stroke', 'Stroke']
plt.figure(figsize=(6,6))
plot_confusion_matrix(cm, species)

In [ ]:
# import matplotlib.pyplot as plt
# for j in range(Results.shape[0]):
#     count = 1
#     for i in range(32):
#         if np.max(imgs_test[j][i])>0:
            
#             plt.figure(figsize=(12,12))
#             plt.subplot(1,3,1)
#             plt.imshow(np.squeeze(imgs_test[j][i]), cmap='gray')
#             plt.title('Original Image')
#             plt.subplot(1,3,2)
#             plt.imshow(np.squeeze(imgs_mask_test[j][i]), cmap='gray')
#             plt.title('Original Mask')
#             plt.subplot(1,3,3)
#             plt.imshow(np.squeeze(Results[j][i]) > .5, cmap='gray')

#             plt.title('Prediction')
#             plt.show()
#             print(valid_path[j][-14:]+"_"+str(count)+"↑")
#             count+=1